## Task Description
Mental Illness Detection


# Setting up the GPU Environment

#### Install Dependencies and Restart Runtime

In [ ]:
!pip install -q transformers
!pip install -q simpletransformers`
!pip install -U imbalanced-learn`

/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file
/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/'CS 7650'

Mounted at /content/drive
/content/drive/MyDrive/CS 7650


# Getting Data

In [ ]:
## Importing packages
import pandas as pd
import numpy as np
import requests
import json
import re

#Feching and storing the data

df = pd.read_excel('reddit_data.xlsx')

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 24,057



,ID,Title,Text,Date,Label,Is_MH
10534,1ftk0yy,PLEASE helpp?,i just moved away for college (i’m 21) and i’m...,2024-10-01 09:32:24,depression,Yes
18954,gx4ic5,"I'm black, I'm scared, tired and I'm at the en...","Jesus, how do I even start. As you could read ...",2020-06-05 13:24:45,depression,Yes
1260,178p3tz,I ate chips now I feel like a failed anorexic,Pretty much what the title says I’ve been scar...,2023-10-15 21:07:38,anorexia,Yes
21199,mdbedu,Accurate description of how living with OCD fe...,NaN,2021-03-25 23:30:07,OCD,Yes
18876,gs4yw8,Does anyone else here have comorbidities? As i...,I'm a gay Christian Latino with BPD (II)\n\nBe...,2020-05-28 12:00:22,bipolar,Yes
12942,1fv1kh7,im so tired,im so stressed all the time. i have no time fo...,2024-10-03 06:35:56,none,No
5658,1fjnfdq,Roma has sacked Daniele De Rossi.,NaN,2024-09-18 07:51:31,none,No
6508,1fmu81w,I can’t stop thinking about it sometimes,My adoptive mom was a extreme control freak an...,2024-09-22 14:12:58,ptsd,Yes
16387,b8t2nw,Would be awesome if they found a cure soon!,NaN,2019-04-03 03:39:36,schizophrenia,Yes
15107,7jeff5,I hope something nice happens to you today.,NaN,2017-12-12 22:12:09,anxiety,Yes


In [ ]:
df['Title'] = df['Title'].fillna('')
df['Text'] = df['Text'].fillna('')
df['Full_Text'] = df['Title'] + ' ' + df['Text']
df['Full_Text'] = df['Full_Text'].astype(str)

In [ ]:
df.head()

,ID,Title,Text,Date,Label,Is_MH,Full_Text
0,100jk9m,The Person Who Traumatized You DMed you on Ins...,The title says it all. I came back home from a...,2023-01-01 13:21:11,ptsd,Yes,The Person Who Traumatized You DMed you on Ins...
1,100t81n,PTSD from antibiotics,I took an antibiotic called ciprofloxacin a fe...,2023-01-01 20:57:25,ptsd,Yes,PTSD from antibiotics I took an antibiotic cal...
2,100xfwl,guys i made a phone call!!!!!!!!,basically my credit card was stolen and loads ...,2023-01-01 23:54:55,anxiety,Yes,guys i made a phone call!!!!!!!! basically my ...
3,100y8dh,The Time when I was Sexually Harassed online (...,*from a user i talked to who doesnt use Reddit...,2023-01-02 00:29:48,ptsd,Yes,The Time when I was Sexually Harassed online (...
4,101kgaa,Having a panic attack due to real event. need ...,TW: POCD. 22F. So I have severe OCD and on med...,2023-01-02 18:59:21,OCD,Yes,Having a panic attack due to real event. need ...


In [ ]:
df_subset1 = df[['ID', 'Full_Text','Label']]
df_subset1 = df_subset1.rename(columns={'Full_Text' : 'text'})
label_mapping = {'none': 0, 'anxiety': 1, 'depression': 2, 'OCD': 3, 'ptsd': 4, 'schizophrenia': 5, 'bipolar': 6, 'anorexia': 7}
df_subset1['labels'] = df_subset1['Label'].map(label_mapping)
df_subset1 = df_subset1.drop(['Label'], axis=1)
df_subset1.head()

,ID,text,labels
0,100jk9m,The Person Who Traumatized You DMed you on Ins...,4
1,100t81n,PTSD from antibiotics I took an antibiotic cal...,4
2,100xfwl,guys i made a phone call!!!!!!!! basically my ...,1
3,100y8dh,The Time when I was Sexually Harassed online (...,4
4,101kgaa,Having a panic attack due to real event. need ...,3


In [ ]:
#Create a function to clean text in the datafrmae
def clean_text(text):

  # Remove emojis
  text = re.sub(r'[:;][\w\d-]*[:;]', '', text)

  # Remove emoticons
  text = re.sub(r'\([^\)]*\)', '', text)

  # Remove hyperlinks
  text = re.sub(r'https?://[^\s]+', '', text)

  # Remove non-English characters
  text = re.sub(r'[^\w\s]', '', text)

  # Convert all text into lowercase
  text = text.lower()

  return text


In [ ]:
df_subset1['text'] = df_subset1['text'].apply(clean_text)
df_subset1.sample(10)

,ID,text,labels
5233,1fh1mi5,great guide by steven miller on astrophography...,0
3651,1ezrdjp,please help my friend is struggling with ana a...,7
12944,1fv1ko8,is there a way out always depressed sad anxiou...,2
10891,1ftt1rg,post egghead do you think there is any old the...,0
3430,1evyp62,names of the androktasiai the androktasiai the...,0
17887,ek3uky,my girlfriend cheated on me with one of my bes...,2
1605,18upvsk,i just deleted someone from my instagram feed ...,1
1631,18xwp8z,i fucked up everything i met my now ex boyfrie...,2
8996,1fsestp,problem with sensations dilemma hi i have phys...,1
11340,1fu3cxd,just got diagnosed your experience with fluoxe...,3


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_subset1, test_size=0.20, random_state=123)

In [ ]:
train_df.head()

,ID,text,labels
15374,8jm8ia,what is it like to have schizophrenia hi every...,5
180,10xcw1t,how do i help someone who has anorexia i just ...,7
21835,ofepru,contamination from pollutants is real and not ...,3
11200,1fu00z6,are books workbooks helpful first off would l...,1
21169,ma4btf,just fucking hug me i sit here on a sunday eve...,1


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19245 entries, 15374 to 19966
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      19245 non-null  object
 1   text    19245 non-null  object
 2   labels  19245 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 601.4+ KB


In [ ]:
train_df['labels'].value_counts()

,count
labels,
0,2773
3,2404
1,2393
2,2375
5,2368
6,2347
4,2332
7,2253


In [ ]:
pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.6 MB/s eta 0:00:00
 

In [ ]:
print(train_df.dtypes)

ID        object
text      object
labels     int64
dtype: object


# RoBERTa

In [ ]:
import os
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 512,
    'num_train_epochs': 4,
    'train_batch_size': 32,
    'fp16': True,
    'output_dir': '/outputs/',
}

In [ ]:
#RoBERTa
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

model = ClassificationModel('roberta', 'roberta-base', num_labels=8, args=train_args)
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Train the model, there is no development or validation set for this dataset
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)
print(result)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

  0%|          | 0/4812 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (808 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (529 > 512). Running this sequence through the model will result in indexing errors


Running Evaluation:   0%|          | 0/62 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


{'mcc': 0.7597248139482171, 'acc': 0.7894846217788861, 'eval_loss': 0.7305357167797704}


In [ ]:
#F1 Score
from sklearn.metrics import f1_score

def f1_macro(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=f1_macro)
print(result)

  0%|          | 0/4812 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (808 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (529 > 512). Running this sequence through the model will result in indexing errors


Running Evaluation:   0%|          | 0/62 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


{'mcc': 0.7597248139482171, 'acc': 0.7867713934371712, 'eval_loss': 0.7305357167797704}


In [ ]:
#Precision and Recall
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.precision_recall_fscore_support)
print(result)

  0%|          | 0/4812 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (808 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (529 > 512). Running this sequence through the model will result in indexing errors


Running Evaluation:   0%|          | 0/62 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


{'mcc': 0.7597248139482171, 'acc': (array([0.96083916, 0.64722222, 0.76660682, 0.74956522, 0.85045045,
       0.75714286, 0.6846543 , 0.90502793]), array([0.94497937, 0.81754386, 0.76114082, 0.74310345, 0.80272109,
       0.72108844, 0.64240506, 0.85865724]), array([0.95284327, 0.72248062, 0.76386404, 0.74632035, 0.82589676,
       0.73867596, 0.66285714, 0.881233  ]), array([727, 570, 561, 580, 588, 588, 632, 566])), 'eval_loss': 0.7305357167797704}


# BERT


In [ ]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 512,
    'num_train_epochs': 4,
    'train_batch_size': 32,
    'fp16': True,
    'output_dir': '/outputs/',
}

In [ ]:
#BERT
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

model2 = ClassificationModel('bert', 'bert-base-uncased', num_labels=8, args=train_args)

# Train the model, there is no development or validation set for this dataset
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model2.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model2.eval_model(test_df, acc=sklearn.metrics.accuracy_score)
print(result)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

  0%|          | 0/4812 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (765 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


Running Evaluation:   0%|          | 0/62 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


{'mcc': 0.7509471560550581, 'acc': 0.7817955112219451, 'eval_loss': 0.8167765270317754}


In [ ]:
#F1 Score
from sklearn.metrics import f1_score

def f1_macro(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro')

result, model_outputs, wrong_predictions = model2.eval_model(test_df, acc=f1_macro)
print(result)

  0%|          | 0/4812 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (765 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


Running Evaluation:   0%|          | 0/62 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


{'mcc': 0.7509471560550581, 'acc': 0.7792615089647632, 'eval_loss': 0.8167765270317754}


In [ ]:
#Precision and Recall
result, model_outputs, wrong_predictions = model2.eval_model(test_df, acc=sklearn.metrics.precision_recall_fscore_support)
print(result)

  0%|          | 0/4812 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (765 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


Running Evaluation:   0%|          | 0/62 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


{'mcc': 0.7509471560550581, 'acc': (array([0.96343179, 0.62482566, 0.74652778, 0.73637961, 0.88157895,
       0.73767606, 0.68844221, 0.88745387]), array([0.94222834, 0.78596491, 0.76648841, 0.72241379, 0.79761905,
       0.71258503, 0.65031646, 0.84982332]), array([0.9527121 , 0.6961927 , 0.75637643, 0.72932985, 0.8375    ,
       0.72491349, 0.66883645, 0.86823105]), array([727, 570, 561, 580, 588, 588, 632, 566])), 'eval_loss': 0.8167765270317754}
